In [1]:
from one.api import ONE
import spikeglx
import numpy as np
from brainbox.io.one import SpikeSortingLoader
from neurodsp.voltage import destripe
import h5py

one = ONE()

pid = 'da8dfec1-d265-44e8-84ce-6ae9c109b8bd'
eid, probe = one.pid2eid(pid)

band = 'ap' # either 'ap','lf'

# Find the relevant datasets and download them
dsets = one.list_datasets(eid, collection=f'raw_ephys_data/{probe}', filename='*.lf.*')
data_files, _ = one.load_datasets(eid, dsets, download_only=True)
bin_file = next(df for df in data_files if df.suffix == '.cbin')

# Use spikeglx reader to read in the whole raw data
sr = spikeglx.Reader(bin_file)
sr.shape

# also get the spikes for this session
ssl = SpikeSortingLoader(pid=pid, one=one)
spikes, clusters, channels = ssl.load_spike_sorting(dataset_types=['spikes.samples'])
clusters = ssl.merge_clusters(spikes, clusters, channels)

c:\Users\Dan\.conda\envs\iblenv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Dan\.conda\envs\iblenv\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\Dan\.conda\envs\iblenv\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
100%|██████████| 10/10.0 [00:03<00:00,  2.76it/s]


In [5]:

f.close()

In [6]:
# go through the raw data a little bit at a time and save it back out
f = h5py.File('./data.hdf5','w')
data = f.create_dataset('destriped', (sr.shape[0], sr.shape[1]-1))

In [7]:
sr.shape

(10739995, 385)

In [11]:
for i in np.arange(0,sr.shape[0], 10000):
    start = i
    stop = i + 99999

    raw = sr[start:stop, :-sr.nsync].T
    data[start:stop,:] = destripe(raw, fs=sr.fs).T

KeyboardInterrupt: 